# Automate scenario runs

The process involves:
1. running a scenario-specific spinup, initialized with `spin3.out.press.pfb` using `scen_spin.tcl` (this will simulate 5 years of 2010 meteorological forcing
2. Running the variable dz subsurface post processing script (python with fortran module, which needs to be installed using: `f2py -c -m postproc scale_pfb.f pfb_read.f writepfb.f` and has dependencies: `scale_pfb.f pfb_read.f writepfb.f`) Need to confirm that subsurface storage is changing by less than 1% between years 4 and 5.
3. Use the last pfb file `slopes_only.out.press.43805.pfb` as the input `spin4.out.press.pfb` (save this in each scen's directory) to the one year simulation
4. Remove all spinup run files to make space
5. Run the 1-year simulation using `scen_run.tcl`
6. Run the full water balance script using `postproc` f2py module. Save the csv dataframe in outputs directory.
7. Remove all run files.

In [1]:
import subprocess
import glob
import os
import shutil
import time

from postproc import scale_pfb
import pandas as pd
from io import StringIO
import itertools
import numpy as np
from shlex import split

%matplotlib inline

In [2]:
# the directory that contains all the subdirectories for each scenario
#scendir_base = '/home/tclim/Dropbox/projects/pf_simple/00preproc/01subsurf_scen'
#scendir_base = '/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific'


# Functions

In [2]:
# Post Processing
def silo2pfb(rundir, bnam, start, stop, fw=0):
    '''
    Converts a timeseries of silo output to pfb format
    (saves converted pfbs to same directory)
    
    bnam     the basename of the files (everything up to the iterating index)
    start    start number of file indexing
    end      end number of file indexing
    fw       fixed width = 0 for non-fixed width iterating index, 
             fixed width = 1 for fixed width (assumed width is 5) index.
    '''
    bashCommand = "tclsh silotopfb_iter_bnam.tcl %s %s %s %s %s" %(rundir, bnam,start,stop, fw)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(output)
    print(error)
    
    # if fw =1, rename the output files to remove $runname.out
    if fw:
        print("renaming output pfb files")
        newbnam = bnam.split(".")[-1]
        
        for i in range(start, stop+1):
            ofnam = "%s/%s.%s.pfb" %(rundir,bnam,i)
            nfnam = "%s/%s.%s.pfb" %(rundir,newbnam,i)
            os.rename(ofnam, nfnam)

def sumoverdomain(rundir, bnam, start, stop):
    '''
    Sums over the entire domain for an input pfb file,
    returns a dataframe with columns for t and the sum values
    
    bnam    the base name of the files to sum
    start   the start number of file indexing
    end     the end number of file indexing
    '''
    bashCommand = "tclsh sum_domain_bnam.tcl %s %s %s %s" %(rundir, bnam,start,stop)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    # Save the output to a dataframe
    output = output.decode("utf-8").format()
    df = pd.read_csv(StringIO(output), sep="\t", header = None)
    df.columns = ['t','sum_val']  # add a header (column name)
    
    # return
    return(df)


# Delete Run Files
def delete_run_files():
    # slopes_only*
    fileList = glob.glob('./slopes_only**', recursive=True)
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)
    
    # .silos
    fileList = glob.glob('./*.silo', recursive=True)
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)
            
    
    # .pfbs
    fileList = glob.glob('./*.pfb', recursive=True)
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)
            
    
    # gp*
    fileList = glob.glob('./gp*', recursive=True)
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)


In [3]:
rundir = '/home/theo/pf_files/pf_machine/scenarios/rundir'
specif_inputs_dir = '/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific'
common_inputs_dir = '/home/theo/pf_files/pf_machine/scenarios/scens_inputs_common'

factors = np.array([[2.0],  # <- bottom
                    [2.0],
                    [2.0],
                    [1.0],
                    [1.0],
                    [1.0],
                    [0.25],
                    [0.25],
                    [0.25],
                    [0.25]]) # <- top

for i in range(9,116):
    scen = 'scen%03d' %i

    print('Now processing... %s' %scen)
    
    # Copy scenario-specific inputs into rundir
    shutil.copy2('%s/%s/mann_scen.pfb' %(specif_inputs_dir, scen), '.')
    shutil.copy2('%s/%s/ind_scen.pfb' %(specif_inputs_dir, scen), '.')
    shutil.copy2('%s/%s/perm_scen.pfb' %(specif_inputs_dir, scen), '.')
    shutil.copy2('%s/%s/drv_vegm.dat' %(specif_inputs_dir, scen), '.')
    
    # Copy scenario-common inputs into rundir
    shutil.copy2('%s/dauphco.nldas.10yr.txt' %common_inputs_dir, '.')
    shutil.copy2('%s/drv_clmin.dat' %common_inputs_dir, '.')
    shutil.copy2('%s/drv_vegp.dat' %common_inputs_dir, '.')
    shutil.copy2('%s/spin3.out.press.pfb' %common_inputs_dir, '.')
    
    print('    Run inputs copied')
    
    #########################################
    # Run scenario-specific spinup (5 years)
    #########################################
    print('    Running scenario-specific spinup')
    start = time.time()
    bashCommand = "tclsh scen_spin.tcl"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    end = time.time()
    print('    Scenario-specific spinup complete.\n    Time Elapsed:  %s s' %(end - start))
    
    
    #########################################
    # Calculate vdz-aware susburface storage
    #########################################
    print('    Calculating vdz-aware subsurface storage for spinup')
    cur_stop = 43805
    
    # create silos needed for subsurf stor
    bashCommand = "tclsh write_subsurf_silos.tcl %s %s %s %s" %(rundir,'slopes_only',0,cur_stop)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(output)
    print(error)
    
    # convert silos to pfbs
    silo2pfb(rundir = rundir, bnam ='subsurface_storage' , start= 0, stop=cur_stop)
    
    # scale pfbs using factors
    for i in range(0,cur_stop+1):
        infnam = '%s/subsurface_storage.%s.pfb' %(rundir, i)
        outfnam = '%s/vdz_subsurface_storage.%s.pfb' %(rundir,i)

        scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
             nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)

    ss = sumoverdomain(rundir = rundir,
                       bnam = 'vdz_subsurface_storage',start = 0, stop= cur_stop)
    print('    vdz-aware subsurface storage calc completed.\n    ss.head(10)\n')
    print(ss.head(10))
    
    # checking subsurf storage stabilized
    print('year-on-year subsurf storage percent change:\n')
    print(ss['sum_val'][list(np.multiply(8610, [0,1,2,3,4,5]))].pct_change())
    
    if (abs(ss['sum_val'][list(np.multiply(8610, [0,1,2,3,4,5]))].pct_change().tail(1)) > 0.01).iloc[0] > 0.01:
        print('    WARNING %s spinup change from year 4 to year 5 NOT LESS THAN 1pc' %scen)
        print('    continuing with next scenario...')
        print('===========================================================')
        break
    
    print('    Spinup stabilization PASSED')
    
    # Save and rename the last pressure file
    print('    saving scenario-specific slopes_only_out.press43805.pfb as %s/%s/spin4.out.press.pfb...' %(specif_inputs_dir, scen))
    os.rename('slopes_only.out.press.43805.pfb', '%s/%s/spin4.out.press.pfb' %(specif_inputs_dir, scen))
    
    # Removing spinup run files to save memory
    print('    Removing all spinup run files...')
    delete_run_files()
    
    ####################################################
    # Run scenario one-year simulation for water balance
    ####################################################
    scen_files = os.listdir('%s/%s' %(specif_inputs_dir, scen))
    for file_name in scen_files:
        full_file_name = os.path.join('%s/%s' %(specif_inputs_dir, scen), file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, '.')
    
    print('    simulation files for %s copied to rundir' %scen)
    
    print('    beginning scenario one-year simulation')
    start = time.time()
    bashCommand = "tclsh scen_run.tcl"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    end = time.time()
    print('    Scenario one-year simulation complete.\n    Time Elapsed:  %s s' %(end - start))
    
    ############################
    # Calculate Water Balance
    ############################
    cur_stop = 8761
    
    # Write silo files for water balance
    bashCommand = "tclsh write_wb_silos.tcl %s %s %s %s" %(rundir,'slopes_only',0,cur_stop)
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    print(output)
    print(error)
    
    # Read in NLDAS input
    nldas = pd.read_fwf('%s/dauphco.nldas.10yr.txt' %rundir, header = None, sep = '')
    nldas.columns = ['DSWR', 'DLWR', 'APCP', 'TMP', 'UGRD', 'VGRD', 'PRESS', 'SPFH']
    print('    Example of NLDAS nldas.head()\n')
    print(nldas.head())
    
    prcp_m_hr = nldas.APCP/1000*60*60 # meters per hour precipitation
    prcp_v = prcp_m_hr*1200*1000 # total precip volume over domain (to use in water balance)
    
    # Overland Flow
    silo2pfb(rundir = rundir, 
         bnam ='slopes_only.out.overlandsum' , start= 1, stop=cur_stop, fw = 1)

    of = sumoverdomain(rundir = rundir, 
                       bnam = 'overlandsum',start = 1, stop= cur_stop)
    of['sum_val'] = of['sum_val']
    print('    Example of overland flow of.head()\n')
    print(of.head(10))
    
    # Surface storage
    silo2pfb(rundir = rundir, bnam ='surface_storage' , start= 0, stop=cur_stop)
    s = sumoverdomain(rundir = rundir, bnam = 'surface_storage',start = 0, stop= cur_stop)
    print('    Example of surface storage s.head()\n')
    print(s.head(10))
    
    # Subsurface storage
    # convert silos to pfbs
    silo2pfb(rundir = rundir, bnam ='subsurface_storage' , start= 0, stop=cur_stop)

    # scale pfbs using factors
    for i in range(0,cur_stop+1):
        infnam = '%s/subsurface_storage.%s.pfb' %(rundir, i)
        outfnam = '%s/vdz_subsurface_storage.%s.pfb' %(rundir,i)

        scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
             nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)

    ss = sumoverdomain(rundir = rundir,
                       bnam = 'vdz_subsurface_storage',start = 0, stop= cur_stop)
    
    print('    Example of subsurface storage ss.head()\n')
    print(ss.head(10))
    
    # Evapotranspiration
    silo2pfb(rundir = rundir, 
         bnam ='slopes_only.out.evaptranssum' , start= 1, stop=cur_stop, fw = 1)

    # scale pfbs using factors
    for i in range(1,cur_stop):
        infnam = '%s/evaptranssum.%s.pfb' %(rundir, i)
        outfnam = '%s/vdz_evaptranssum.%s.pfb' %(rundir,i)

        scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
             nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)

    et = sumoverdomain(rundir = rundir, 
                       bnam = 'vdz_evaptranssum',start = 1, stop= cur_stop)
    et['sum_val'] = et['sum_val']
    et['t'] = et['t'] - 1
    print('    Example of evapotranspiration et.head()\n')
    print(et.head(10))
    
    
    # Put pieces of water balance together
    print('    Putting pieces of water balance together')
    wb = ss.merge(of, on = 't', how = 'left').merge(s, on = 't', how = 'left').merge(et, on = 't', how = 'left')
    wb.columns = ['t', 'vdz_subsurfstor', 'overland_flow', 'surf_stor', 'vdz_et']
    wb['overland_flow'] = (- wb['overland_flow'])  
    wb['overland_flow'] = wb['overland_flow'].shift(-1)  
    wb['dom_tot'] = wb['vdz_subsurfstor'] +  wb['surf_stor'] 
    wb['bndy_flux'] = wb['vdz_et'] + wb['overland_flow']                
    wb['exp_wb'] = wb['dom_tot'].shift(-1) + wb['bndy_flux']
    wb['dom_tot_chg'] = wb['dom_tot'].diff()
    wb['dom_tot_chg'] = wb['dom_tot_chg'].shift(-1)
    wb['diff'] = wb['dom_tot_chg'] - wb['bndy_flux']
    wb['pc_diff'] = abs(wb['diff'])/wb['exp_wb']*100
    wb['prcp_v'] = prcp_v[:cur_stop]

    print('    Example of wb.head()\n')
    print(wb.head(20))

    # check what the pct differences are
    if abs(max(wb.pc_diff.dropna())) > 0.0005:
        print('    WARNING %s water balance not closing' %scen)
        print('    continuing with next scenario...')
        print('===========================================================')
        break
    
    # Save the water balance
    wb.to_csv('/home/theo/pf_files/pf_machine/scenarios/wb_outputs/wb_%s.csv' %scen)
    
    # Remove run files to save memory
    print('    Removing all simulation run files...')
    delete_run_files()
    
    
    print('===========================================================')

Now processing... scen009
    Run inputs copied
    Running scenario-specific spinup
    Scenario-specific spinup complete.
    Time Elapsed:  3126.6695125102997 s
    Calculating vdz-aware subsurface storage for spinup
b''
None
b''
None
    vdz-aware subsurface storage calc completed.
    ss.head(10)

   t       sum_val
0  0  3.229581e+06
1  1  3.229986e+06
2  2  3.230307e+06
3  3  3.230718e+06
4  4  3.230780e+06
5  5  3.230802e+06
6  6  3.230731e+06
7  7  3.230628e+06
8  8  3.230525e+06
9  9  3.230423e+06
year-on-year subsurf storage percent change:

0             NaN
8610    -0.014511
17220    0.001593
25830    0.004654
34440    0.001705
43050   -0.005087
Name: sum_val, dtype: float64
    Spinup stabilization PASSED
    saving scenario-specific slopes_only_out.press43805.pfb as /home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen009/spin4.out.press.pfb...
    Removing all spinup run files...
Error while deleting file :  ./slopes_only.out
    simulation files for scen0

b''
None
    Example of surface storage s.head()

   t       sum_val
0  0  95373.249711
1  1  95427.507467
2  2  95469.101928
3  3  95526.619256
4  4  95534.573900
5  5  95535.271311
6  6  95521.186136
7  7  95501.600333
8  8  95481.970128
9  9  95462.707709
b''
None
    Example of subsurface storage ss.head()

   t       sum_val
0  0  3.136601e+06
1  1  3.137124e+06
2  2  3.137515e+06
3  3  3.138007e+06
4  4  3.138110e+06
5  5  3.138171e+06
6  6  3.138131e+06
7  7  3.138055e+06
8  8  3.137981e+06
9  9  3.137907e+06
b''
None
renaming output pfb files
    Example of evapotranspiration et.head()

   t     sum_val
0  0  652.877604
1  1  507.827201
2  2  624.537862
3  3  187.180009
4  4  136.618399
5  5   21.196060
6  6  -19.773969
7  7  -18.487248
8  8  -17.685392
9  9  -16.959757
    Putting pieces of water balance together
    Example of wb.head()

     t  vdz_subsurfstor  overland_flow     surf_stor      vdz_et  \
0    0     3.136601e+06     -75.360730  95373.249711  652.877604   
1   

KeyboardInterrupt: 

# TESTING

# Copy scenario-specific inputs into run directory

In [5]:
scen = 'scen000'

In [6]:
[print (f) for f in glob.glob('%s/%s/*' %(scendir_base, scen))]


/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/mann_scen.pfb
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/spin4.out.press.pfb
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/perm_scen.pfb
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/drv_vegm.dat
/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/ind_scen.pfb


[None, None, None, None, None]

In [7]:
'%s/%s/*' %(scendir_base, scen)

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_specific/scen000/*'

In [13]:
shutil.copy2('%s/%s/mann_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/ind_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/perm_scen.pfb' %(scendir_base, scen), '.')
shutil.copy2('%s/%s/drv_vegm.dat' %(scendir_base, scen), '.')

'./drv_vegm.dat'

# Copy scenario-common files into run directory 

In [14]:
shutil.copy2('%s/dauphco.nldas.10yr.txt' %clm_inputs_dir, '.')
shutil.copy2('%s/drv_clmin.dat' %clm_inputs_dir, '.')
shutil.copy2('%s/drv_vegp.dat' %clm_inputs_dir, '.')

'./drv_vegp.dat'

# Run Scenario-specific spinup

multiple core test (for 1000 timesteps):

1,1,1: 57 s
2,2,1: 52 s
4,4,1: 59 s

multiple core test (for 5000 timesteps):

1,1,1: 401 s
2,2,1: 325 s
4,4,1: 394 s
4,2,1: 342 s

In [20]:
start = time.time()
bashCommand = "tclsh scen_spin.tcl"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Time Elapsed:  342.6045961380005 s


In [21]:
print(output.decode("utf-8").format())






ParFlow run complete
Using process grid (4,2,1)
Using process grid (4,2,1)
Using process grid (4,2,1)
Using process grid (4,2,1)



# Calculate vdz_subsurf storage

In [23]:
rundir = '/home/theo/pf_files/pf_machine/scenarios/rundir'
#cur_stop = 10
cur_stop = 43805 # five years of scenario-specific spinup

In [24]:
# create silos needed for subsurf stor
bashCommand = "tclsh write_subsurf_silos.tcl %s %s %s %s" %(rundir,'slopes_only',0,cur_stop)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output)
print(error)

b''
None


In [25]:
factors = np.array([[2.0],  # <- bottom
                    [2.0],
                    [2.0],
                    [1.0],
                    [1.0],
                    [1.0],
                    [0.25],
                    [0.25],
                    [0.25],
                    [0.25]]) # <- top

In [26]:

# convert silos to pfbs
silo2pfb(rundir = rundir, bnam ='subsurface_storage' , start= 0, stop=cur_stop)


b''
None


In [27]:
# scale pfbs using factors
for i in range(0,cur_stop+1):
    infnam = '%s/subsurface_storage.%s.pfb' %(rundir, i)
    outfnam = '%s/vdz_subsurface_storage.%s.pfb' %(rundir,i)
    
    scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
         nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)
    
ss = sumoverdomain(rundir = rundir,
                   bnam = 'vdz_subsurface_storage',start = 0, stop= cur_stop)
ss.head(10)


,t,sum_val
0,0,3.261884e+06
1,1,3.262279e+06
2,2,3.262577e+06
3,3,3.262965e+06
4,4,3.263002e+06
5,5,3.263000e+06
6,6,3.262907e+06
7,7,3.262780e+06
8,8,3.262653e+06
9,9,3.262526e+06


### Check each year's volume pc change

In [15]:
print(ss['sum_val'][list(np.multiply(8610, [0,1,2,3,4,5]))].pct_change())

0             NaN
8610     0.010764
17220    0.012395
25830    0.007813
34440    0.002453
43050   -0.004638
Name: sum_val, dtype: float64

# Save and rename the last pressure file
Put it in the scenario directory

In [20]:
'%s/%s/spin4.out.press.pfb' %(scendir_base, scen)

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000/mann_scen.pfb'

In [21]:
os.rename('slopes_only.out.press.43805.pfb', '%s/%s/spin4.out.press.pfb' %(scendir_base, scen))

# Remove memory-eating runtime files

This is the bash script:

```
find . -name "slopes_only*" -print0 | xargs -0 rm
find . -name "*silo" -print0 | xargs -0 rm
find . -name "*pfb" -print0 | xargs -0 rm
rm gp*
```

In [25]:
delete_run_files()

Removed slopes_only*
Time Elapsed:  0.02194666862487793 s


# Re-copy input files scenario directory

This should include the original subsurface pfbs, and spin4.out.press.pfb from the scenario-specific spinup

In [41]:
scendir_base
scen
scendir = '%s/%s' %(scendir_base, scen)
scendir

'/home/theo/pf_files/pf_machine/scenarios/scens_inputs_subsurf/scen000'

In [43]:
scen_files

['mann_scen.pfb', 'spin4.out.press.pfb', 'perm_scen.pfb', 'ind_scen.pfb']

In [44]:
scen_files = os.listdir(scendir)
for file_name in scen_files:
    full_file_name = os.path.join(scendir, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, '.')

# Run one year

In [48]:
start = time.time()
bashCommand = "tclsh scen_run.tcl"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
end = time.time()
print('Time Elapsed:  %s s' %(end - start))

Time Elapsed:  705.4529099464417 s


In [47]:
print(output.decode("utf-8").format())

# Calculate Water Balance

In [51]:
cur_stop = 8761

##  Write Silo Files for Water Balance

In [54]:
bashCommand = "tclsh write_wb_silos.tcl %s %s %s %s" %(rundir,'slopes_only',0,cur_stop)
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output)
print(error)

b''
None


## Read in NLDAS input 

In [61]:
nldas = pd.read_fwf('%s/dauphco.nldas.10yr.txt' %rundir, header = None, sep = '')
nldas.columns = ['DSWR', 'DLWR', 'APCP', 'TMP', 'UGRD', 'VGRD', 'PRESS', 'SPFH']
nldas.head()
prcp_m_hr = nldas.APCP/1000*60*60 # meters per hour precipitation
prcp_v = prcp_m_hr*1200*1000 # total precip volume over domain (to use in water balance)

## Overland Flow 

In [56]:
silo2pfb(rundir = rundir, 
         bnam ='slopes_only.out.overlandsum' , start= 1, stop=cur_stop, fw = 1)

of = sumoverdomain(rundir = rundir, 
                   bnam = 'overlandsum',start = 1, stop= cur_stop)
of['sum_val'] = of['sum_val']

of.head(10)

b''
None
renaming output pfb files


,t,sum_val
0,1,98.036517
1,2,112.069583
2,3,128.935113
3,4,106.983865
4,5,92.887362
5,6,78.819071
6,7,69.332908
7,8,64.372935
8,9,61.696368
9,10,60.232115


## Surface Storage

In [57]:
silo2pfb(rundir = rundir, bnam ='surface_storage' , start= 0, stop=cur_stop)
s = sumoverdomain(rundir = rundir, bnam = 'surface_storage',start = 0, stop= cur_stop)
s.head(10)

b''
None


,t,sum_val
0,0,62271.597645
1,1,62360.658559
2,2,62409.543978
3,3,62473.282800
4,4,62460.089782
5,5,62448.830927
6,6,62425.894149
7,7,62404.894822
8,8,62388.990196
9,9,62376.030953


##  Subsurface storage

In [58]:
# convert silos to pfbs
silo2pfb(rundir = rundir, bnam ='subsurface_storage' , start= 0, stop=cur_stop)

# scale pfbs using factors
for i in range(0,cur_stop+1):
    infnam = '%s/subsurface_storage.%s.pfb' %(rundir, i)
    outfnam = '%s/vdz_subsurface_storage.%s.pfb' %(rundir,i)
    
    scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
         nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)
    
ss = sumoverdomain(rundir = rundir,
                   bnam = 'vdz_subsurface_storage',start = 0, stop= cur_stop)
ss.head(10)

b''
None


,t,sum_val
0,0,3.351440e+06
1,1,3.351910e+06
2,2,3.352259e+06
3,3,3.352693e+06
4,4,3.352788e+06
5,5,3.352844e+06
6,6,3.352811e+06
7,7,3.352744e+06
8,8,3.352678e+06
9,9,3.352613e+06


## Evapotranspiration

In [59]:
silo2pfb(rundir = rundir, 
         bnam ='slopes_only.out.evaptranssum' , start= 1, stop=cur_stop, fw = 1)

# scale pfbs using factors
for i in range(1,cur_stop):
    infnam = '%s/evaptranssum.%s.pfb' %(rundir, i)
    outfnam = '%s/vdz_evaptranssum.%s.pfb' %(rundir,i)
    
    scale_pfb(pfbinfnam = infnam, vdzarr = factors, pfboutfnam = outfnam, 
         nx = 12, ny = 10, dx = 10, dy = 10, dz = 1)

et = sumoverdomain(rundir = rundir, 
                   bnam = 'vdz_evaptranssum',start = 1, stop= cur_stop)
et['sum_val'] = et['sum_val']
et['t'] = et['t'] - 1
et.head(10)

b''
None
renaming output pfb files


,t,sum_val
0,0,657.213253
1,1,510.364931
2,2,626.381574
3,3,188.641342
4,4,138.005265
5,5,22.480711
6,6,-18.516468
7,7,-17.223734
8,8,-16.457629
9,9,-15.779504


## Put pieces of WB together


In [63]:
wb = ss.merge(of, on = 't', how = 'left').merge(s, on = 't', how = 'left').merge(et, on = 't', how = 'left')
wb.columns = ['t', 'vdz_subsurfstor', 'overland_flow', 'surf_stor', 'vdz_et']
wb['overland_flow'] = (- wb['overland_flow'])  
wb['overland_flow'] = wb['overland_flow'].shift(-1)  
wb['dom_tot'] = wb['vdz_subsurfstor'] +  wb['surf_stor'] 
wb['bndy_flux'] = wb['vdz_et'] + wb['overland_flow']                
wb['exp_wb'] = wb['dom_tot'].shift(-1) + wb['bndy_flux']
wb['dom_tot_chg'] = wb['dom_tot'].diff()
wb['dom_tot_chg'] = wb['dom_tot_chg'].shift(-1)
wb['diff'] = wb['dom_tot_chg'] - wb['bndy_flux']
wb['pc_diff'] = abs(wb['diff'])/wb['exp_wb']*100
wb['prcp_v'] = prcp_v[:cur_stop]


wb.head(20)

# check what the pct differences are
max(wb.pc_diff.dropna())

1.0261088922071804e-07

## Save wb csv

In [65]:
wb.to_csv('/home/theo/pf_files/pf_machine/scenarios/wb_outputs/wb_%s.csv' %scen)

# Remove run files

In [ ]:
delete_run_files()